 **Model Deployment L2**

In [ ]:
# pip install colabcode

In [ ]:
# pip install fastapi

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import pickle

In [24]:
df=pd.read_csv('gdrive/My Drive/Project_Work_DS/New/df_L2.csv')

In [25]:
df.head()

,L2LoginID,L2Decision,L2_Op_Time,L2_OP_PER
0,32661,Time out,27,bad
1,32661,Accept,13,good
2,32153,Accept,5,excellent
3,32153,Accept,11,good
4,32153,Accept,5,excellent


In [26]:
from sklearn.preprocessing import LabelEncoder
#Applying Label Encoding to column L2Decision
lb = LabelEncoder()
df["L2Decision"] = lb.fit_transform(df["L2Decision"])

In [27]:
df.head()

,L2LoginID,L2Decision,L2_Op_Time,L2_OP_PER
0,32661,2,27,bad
1,32661,0,13,good
2,32153,0,5,excellent
3,32153,0,11,good
4,32153,0,5,excellent


In [28]:
X = df.drop(['L2_OP_PER'], axis=1)
y = df.L2_OP_PER

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)

In [31]:
tree =BaggingClassifier(DecisionTreeClassifier(),n_estimators=50,max_samples=0.5,max_features=1.0)

In [32]:
tree.fit(X_train, y_train)

Pkl_Filename = "model_tree.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(tree, file)

In [33]:
from pydantic import BaseModel

class OperatorL2(BaseModel):
    L2LoginID: int 
    L2Decision: int 
    L2_Op_Time: int 
    class Config:
        schema_extra = {
            "example": {
                "L2LoginID": 32661, 
                "L2Decision": 2, 
                "L2_Op_Time": 27,
              }
        }

In [34]:
from fastapi import FastAPI
import pickle

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_tree.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/predict')
def get_music_category(data: OperatorL2):
    received = data.dict()
    L2LoginID = received['L2LoginID']
    L2Decision = received['L2Decision']
    L2_Op_Time = received['L2_Op_Time']
    pred_name = model.predict([[L2LoginID, L2Decision, L2_Op_Time]]).tolist()[0]
    return {'prediction': pred_name}

In [35]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [ ]:
server.run_app(app=app)

Public URL: NgrokTunnel: "https://cdb6c18d443f.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [2198]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     157.33.43.106:0 - "GET / HTTP/1.1" 200 OK
INFO:     157.33.43.106:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     157.33.43.106:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     157.33.43.106:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     157.33.43.106:0 - "POST /predict HTTP/1.1" 200 OK
